# Demo: Apply trained VCD-Net model  
This notebook demonstrates applying a VCD-Net  model for a light field reconstruction.


## Examine the parameters
We define all parameters in a [configuration script config.py](./config.py). Make the edit according to specific application and dataset before running this notebook.  

Typical parameters include:
- **PSF.n_slices** : number of z slices of targeted 3-D reconstruction
- **PSF.Nnum**     : number of pixels behind each lenslet
- **label**        : label of the pretrained model to be used for inference
- **VALID.lf2d_path**     : folder where the 2D light field images to be reconstructed are stored
- **VALID.saving_path**   : folder where the reconstruced results will be stored

In [1]:
from evalMB import *
import warnings
warnings.filterwarnings('ignore')
import glob


print("Parameters defined in config.py:")
print("PSF related: ")
for par, val in config.PSF.items():
    print('    {:<30}   {:<30}'.format(par,val))
        
print("Prediction related: ")
for par, val in config.VALID.items():
    print('    {:<30}   {:<30}'.format(par,val))




Parameters defined in config.py:
PSF related: 
    n_slices                         61                            
    Nnum                             13                            
Prediction related: 
    ckpt_dir                         /Users/timothyspellman/Dropbox/Python/VCD-Net/vcd-example-data/checkpoint/tubulin_10x06_N13_OSR3/
    lf2d_path                        /Users/timothyspellman/Dropbox/Python/VCD-Net/vcd-example-data/data/tubesMacbook_Nnum13OSR3_LF/
    saving_path                      /Users/timothyspellman/Dropbox/Python/VCD-Net/vcd-example-data/data/tubesMacbook_Nnum13OSR3_Results/


In [2]:
ckpt = 0 #0 means latest
batch_size = 1
use_cpu = 1                                             
infer(ckpt, batch_size=batch_size, use_cpu=use_cpu)

[TL] [*] creates /Users/timothyspellman/Dropbox/Python/VCD-Net/vcd-example-data/data/tubesMacbook_Nnum13OSR3_Results/ ...


FileNotFoundError: [Errno 2] No such file or directory: '/Users/timothyspellman/Dropbox/Python/VCD-Net/vcd-example-data/data/tubesMacbook_Nnum13OSR3_LF/'

## Plot the results

In [ ]:
import ipyvolume as ipv

test_file = 'vcd-001-000009.tif'
recon = get_and_rearrange3d(test_file, config.VALID.saving_path, normalize)
# recons=np.ndarray([recon.shape[0],recon.shape[1],recon.shape[2],3])

f=ipv.quickvolshow(recon)
ipv.show()

# ##threshold
# a,b,c=np.where(recon>=0.1)
# a=np.asarray(a,dtype='float64')
# b=np.asarray(b,dtype='float64')
# c=np.asarray(c,dtype='float64')

# ##scatter
# # f=ipv.quickscatter(a,b,c)
# ipv.show()

# ##surface
# from scipy.spatial import ConvexHull, convex_hull_plot_2d
# points=np.concatenate((np.expand_dims(a,axis=1),np.expand_dims(b,axis=1),np.expand_dims(c,axis=1)),axis=1)
# hull = ConvexHull(points)
# f=ipv.plot_trisurf(a,b,c, triangles=hull.simplices, color='red') ##surface
# ipv.show()

In [ ]:
##bokeh embeds ipv figures as html exceeds the char limit for squarespace 
import ipyvolume
import ipyvolume as ipv
import vaex
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import CustomJS, ColumnDataSource
import ipyvolume.bokeh
from bokeh.resources import CDN
from bokeh.embed import components
import ipywidgets

ds = vaex.example()
N = 5

ipv.figure()
quiver = ipv.quiver(ds.data.x[:N],  ds.data.y[:N],  ds.data.z[:N],
                    ds.data.vx[:N], ds.data.vy[:N], ds.data.vz[:N],
                    size=1, size_selected=5, color_selected="grey")
ipv.xyzlim(-30, 30)
ipv.show()

output_notebook()

xin=ds.data.Lz[:N]
yin=ds.data.E[:N]
data_source = ColumnDataSource(data=dict(x=xin.astype('uint8'), y=yin.astype('uint8')))
p = figure(title="E Lz space", tools='lasso_select', width=500, height=500)
r = p.circle('x', 'y', source=data_source, color="navy", alpha=0.2)
ipyvolume.bokeh.link_data_source_selection_to_widget(data_source, quiver, 'selected')
show(p)

out = ipywidgets.Output()
with out:
    show(p)
ipywidgets.HBox([out, ipv.gcc()])

script, div = components((p))
template_options = dict(extra_script_head=script + CDN.render_js() + CDN.render_css(),
                        body_pre="<h2>Do selections in 2d (bokeh)<h2>" + div + "<h2>And see the selection in ipyvolume<h2>")
figCode=ipyvolume.embed.embed_html("tmp/bokeh.html",
                           [ipv.gcc(), ipyvolume.bokeh.wmh], all_states=True,
                           template_options=template_options)

!open tmp/bokeh.html

In [ ]:
imageio.volread

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import gridspec
from configMB import config
import imageio
import numpy as np
import mpld3


test_file = 'vcd-001-000000.tif'
# recon = get_and_rearrange3d(test_file, config.VALID.saving_path, normalize)
recon = imageio.volread(config.VALID.saving_path+test_file)
print(recon.shape)
fig=plt.figure(figsize=(8,8))
gs = gridspec.GridSpec(2, 1, height_ratios=[recon.shape[0],recon.shape[2]*1/0.34]) 
xy_view = plt.subplot(gs[0])
xy_view.imshow(np.sum(recon, axis=-1, keepdims=False),cmap='hot')
xy_view.set_title('VCD-XY')
plt.axis('off')
xz_view = plt.subplot(gs[1])
xz_view.imshow(np.transpose(np.sum(recon, axis=0, keepdims=False)),cmap='hot',aspect=1/0.34)
xz_view.set_title('VCD-XZ')
plt.axis('off')
plt.show()

##create html file
html_str = mpld3.fig_to_html(fig)
Html_file= open("index.html","w")
Html_file.write(html_str)
Html_file.close()